<a href="https://colab.research.google.com/github/aadhamashraf/Intent-Based-Network-Generation-Augmentation/blob/main/laajresultsanalysis_converted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import pprint

df = pd.read_excel("/content/gen_dataset_evaluation.xlsx" , sheet_name=1)
df.columns = df.columns.str.strip()
df["Intent Type"] = df["Intent Type"].str.strip()
df = df[df["Intent Type"] != ":---"]

df.columns

numericalValues  = ['Standards Alignment / Correctness (1-5)' , 'Complexity (1-5)' , 'Readability (1-5)' , 'Relevance (1-5)' , 'Coherence (1-5)' , 'Consistency (1-5)' , 'Completeness (1-5)' , 'Overall Score']

for i in numericalValues:
    df[i] = pd.to_numeric(df[i], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 0 to 5004
Data columns (total 19 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Intent Type                              5000 non-null   object 
 1   Description                              5000 non-null   object 
 2   Standards Alignment / Correctness (1-5)  5000 non-null   int64  
 3   Standards Justification                  5000 non-null   object 
 4   Complexity (1-5)                         5000 non-null   int64  
 5   Complexity Justification                 5000 non-null   object 
 6   Readability (1-5)                        5000 non-null   int64  
 7   Readability Justification                5000 non-null   object 
 8   Relevance (1-5)                          5000 non-null   int64  
 9   Relevance Justification                  5000 non-null   object 
 10  Coherence (1-5)                          5000 non-nul

# **Low-Score Distribution by Evaluation Dimension**

In [90]:
results = []

for col in numericalValues:
    total = df[col].notna().sum()
    below3 = (df[col] < 4).sum()
    perc = (below3 / total * 100) if total > 0 else 0
    results.append([col, below3, total, round(perc, 2)])

summary_df = pd.DataFrame(results, columns=["Column", "Count < 4", "Total", "Percentage %"])
print(summary_df)


                                    Column  Count < 4  Total  Percentage %
0  Standards Alignment / Correctness (1-5)        306   5000          6.12
1                         Complexity (1-5)       1194   5000         23.88
2                        Readability (1-5)        292   5000          5.84
3                          Relevance (1-5)         46   5000          0.92
4                          Coherence (1-5)        296   5000          5.92
5                        Consistency (1-5)        374   5000          7.48
6                       Completeness (1-5)        636   5000         12.72
7                            Overall Score        313   5000          6.26


In [91]:
all_summaries = []

for col in numericalValues:
    intent_counts = (df[df[col] < 4].groupby("Intent Type").size().reindex(df["Intent Type"].unique(), fill_value=0) )

    total = df[col].notna().sum()
    below4 = (df[col] < 4).sum()
    perc = (below4 / total * 100) if total > 0 else 0

    row = {"Metric": col, "Total <4": below4, "% <4": round(perc, 2)}
    row.update(intent_counts.to_dict())
    all_summaries.append(row)

intent_summary_df = pd.DataFrame(all_summaries)
intent_summary_df

,Metric,Total <4,% <4,Regular Notification Request,Intent Report Request,Deployment Intent,Intent Feasibility Check,Performance Assurance Intent,Modification Intent
0,Standards Alignment / Correctness (1-5),306,6.12,116,10,37,31,65,47
1,Complexity (1-5),1194,23.88,284,148,137,272,161,192
2,Readability (1-5),292,5.84,123,0,43,8,74,44
3,Relevance (1-5),46,0.92,5,0,28,0,12,1
4,Coherence (1-5),296,5.92,189,0,36,1,14,56
5,Consistency (1-5),374,7.48,145,4,76,22,66,61
6,Completeness (1-5),636,12.72,172,10,60,145,134,115
7,Overall Score,313,6.26,149,0,27,8,50,79


In [92]:
bins = [0, 1, 2, 3, 4]
labels = ["0–1", "1–2", "2–3", "3–4"]

intent_summary_dfs = {}

for intent in df["Intent Type"].unique():
    all_summaries = []

    df_intent = df[df["Intent Type"] == intent]

    for col in numericalValues:
        total = df_intent[col].notna().sum()

        binned = pd.cut(df_intent[col], bins=bins, labels=labels, right=False)
        range_counts = binned.value_counts().reindex(labels, fill_value=0)
        range_perc = (range_counts / total * 100).round(2) if total > 0 else 0

        row = {"Metric": col, "Total": total}
        for label in labels:
            row[f"Count {label}"] = range_counts[label]
            row[f"% {label}"] = range_perc[label]

        all_summaries.append(row)

    intent_summary_dfs[intent] = pd.DataFrame(all_summaries)

In [23]:
deployment_summary = intent_summary_dfs.get('Deployment Intent')
deployment_summary

,Metric,Total,Count 0–1,% 0–1,Count 1–2,% 1–2,Count 2–3,% 2–3,Count 3–4,% 3–4
0,Standards Alignment / Correctness (1-5),826,0,0.0,0,0.00,3,0.36,34,4.12
1,Complexity (1-5),826,0,0.0,0,0.00,19,2.30,118,14.29
2,Readability (1-5),826,0,0.0,10,1.21,16,1.94,17,2.06
3,Relevance (1-5),826,0,0.0,3,0.36,3,0.36,22,2.66
4,Coherence (1-5),826,0,0.0,3,0.36,4,0.48,29,3.51
5,Consistency (1-5),826,0,0.0,9,1.09,38,4.60,29,3.51
6,Completeness (1-5),826,0,0.0,0,0.00,6,0.73,54,6.54
7,Overall Score,826,0,0.0,0,0.00,0,0.00,27,3.27


In [24]:
feasability_check_summary = intent_summary_dfs.get('Intent Feasibility Check')
feasability_check_summary

,Metric,Total,Count 0–1,% 0–1,Count 1–2,% 1–2,Count 2–3,% 2–3,Count 3–4,% 3–4
0,Standards Alignment / Correctness (1-5),843,0,0.0,0,0.0,1,0.12,30,3.56
1,Complexity (1-5),843,0,0.0,0,0.0,0,0.00,272,32.27
2,Readability (1-5),843,0,0.0,0,0.0,1,0.12,7,0.83
3,Relevance (1-5),843,0,0.0,0,0.0,0,0.00,0,0.00
4,Coherence (1-5),843,0,0.0,0,0.0,0,0.00,1,0.12
5,Consistency (1-5),843,0,0.0,0,0.0,3,0.36,19,2.25
6,Completeness (1-5),843,0,0.0,0,0.0,14,1.66,131,15.54
7,Overall Score,843,0,0.0,0,0.0,0,0.00,8,0.95


In [25]:
report_request_summary = intent_summary_dfs.get('Intent Report Request')
report_request_summary

,Metric,Total,Count 0–1,% 0–1,Count 1–2,% 1–2,Count 2–3,% 2–3,Count 3–4,% 3–4
0,Standards Alignment / Correctness (1-5),836,0,0.0,0,0.0,0,0.00,10,1.20
1,Complexity (1-5),836,0,0.0,0,0.0,0,0.00,148,17.70
2,Readability (1-5),836,0,0.0,0,0.0,0,0.00,0,0.00
3,Relevance (1-5),836,0,0.0,0,0.0,0,0.00,0,0.00
4,Coherence (1-5),836,0,0.0,0,0.0,0,0.00,0,0.00
5,Consistency (1-5),836,0,0.0,0,0.0,0,0.00,4,0.48
6,Completeness (1-5),836,0,0.0,0,0.0,1,0.12,9,1.08
7,Overall Score,836,0,0.0,0,0.0,0,0.00,0,0.00


In [26]:
modification_summary = intent_summary_dfs.get('Modification Intent')
modification_summary

,Metric,Total,Count 0–1,% 0–1,Count 1–2,% 1–2,Count 2–3,% 2–3,Count 3–4,% 3–4
0,Standards Alignment / Correctness (1-5),811,0,0.0,0,0.00,0,0.00,47,5.80
1,Complexity (1-5),811,0,0.0,0,0.00,2,0.25,190,23.43
2,Readability (1-5),811,0,0.0,0,0.00,5,0.62,39,4.81
3,Relevance (1-5),811,0,0.0,1,0.12,0,0.00,0,0.00
4,Coherence (1-5),811,0,0.0,0,0.00,18,2.22,38,4.69
5,Consistency (1-5),811,0,0.0,4,0.49,16,1.97,41,5.06
6,Completeness (1-5),811,0,0.0,0,0.00,20,2.47,95,11.71
7,Overall Score,811,0,0.0,0,0.00,2,0.25,77,9.49


In [27]:
regular_notification_request_summary = intent_summary_dfs.get('Regular Notification Request')
regular_notification_request_summary

,Metric,Total,Count 0–1,% 0–1,Count 1–2,% 1–2,Count 2–3,% 2–3,Count 3–4,% 3–4
0,Standards Alignment / Correctness (1-5),836,0,0.0,0,0.00,34,4.07,82,9.81
1,Complexity (1-5),836,0,0.0,0,0.00,0,0.00,284,33.97
2,Readability (1-5),836,0,0.0,0,0.00,11,1.32,112,13.40
3,Relevance (1-5),836,0,0.0,0,0.00,0,0.00,5,0.60
4,Coherence (1-5),836,0,0.0,0,0.00,8,0.96,181,21.65
5,Consistency (1-5),836,0,0.0,6,0.72,37,4.43,102,12.20
6,Completeness (1-5),836,0,0.0,1,0.12,3,0.36,168,20.10
7,Overall Score,836,0,0.0,0,0.00,0,0.00,149,17.82


In [28]:
performance_assurance_summary = intent_summary_dfs.get('Performance Assurance Intent')
performance_assurance_summary

,Metric,Total,Count 0–1,% 0–1,Count 1–2,% 1–2,Count 2–3,% 2–3,Count 3–4,% 3–4
0,Standards Alignment / Correctness (1-5),848,0,0.0,0,0.00,16,1.89,49,5.78
1,Complexity (1-5),848,0,0.0,0,0.00,8,0.94,153,18.04
2,Readability (1-5),848,0,0.0,1,0.12,16,1.89,57,6.72
3,Relevance (1-5),848,0,0.0,2,0.24,1,0.12,9,1.06
4,Coherence (1-5),848,0,0.0,0,0.00,3,0.35,11,1.30
5,Consistency (1-5),848,0,0.0,6,0.71,23,2.71,37,4.36
6,Completeness (1-5),848,0,0.0,7,0.83,44,5.19,83,9.79
7,Overall Score,848,0,0.0,0,0.00,3,0.35,47,5.54


# **Intent Types Analysis**

In [80]:
justification_map = {
    "Standards Alignment / Correctness (1-5)": "Standards Justification",
    "Complexity (1-5)": "Complexity Justification",
    "Readability (1-5)": "Readability Justification",
    "Relevance (1-5)": "Relevance Justification",
    "Coherence (1-5)": "Coherence Justification",
    "Consistency (1-5)": "Consistency Justification",
    "Completeness (1-5)": "Completeness Justification",
    "Overall Score": "Overall Comments"
}

def get_intent_justification_details(df, intent_type, score_name):
    """
    Extract unique justifications for a given intent type and score/metric.

    Args:
        df (pd.DataFrame): The full DataFrame containing scores and justifications.
        intent_type (str): The intent type to filter (e.g., "Deployment Intent").
        score_name (str): The metric/score column name (e.g., "Complexity (1-5)").

    Returns:
        dict: Dictionary with binned ranges as keys.
              Each range contains unique justifications with their count and row indices.

    Example output:
    {
        '0–1': {
            'Justification A': {'count': 2, 'indices': [10, 25]},
            'Justification B': {'count': 1, 'indices': [42]}
        },
        '1–2': {...},
        '2–3': {...},
        '3–4': {...}
    }
    """
    df = df.copy()
    df.columns = df.columns.str.strip()
    df["Intent Type"] = df["Intent Type"].str.strip()

    df_intent = df[df["Intent Type"] == intent_type]

    justification_col = justification_map.get(score_name)
    if justification_col is None:
        raise ValueError(f"No justification column found for score '{score_name}'")

    binned = pd.cut(df_intent[score_name], bins=bins, labels=labels, right=False)

    details_per_range = {}
    for label in labels:
        subset = df_intent.loc[binned == label, justification_col].dropna()
        unique_justifications = subset.unique()
        just_detail = {}
        for just in unique_justifications:
            indices = subset[subset == just].index.tolist()
            just_detail[just] = {"count": len(indices), "indices": indices}
        details_per_range[label] = just_detail

    return details_per_range

## Intent Feasibility Check Analysis


### **Complexity Measure (1-5)**

In [81]:
feasibility_complexity = get_intent_justification_details(df, "Intent Feasibility Check", "Complexity (1-5)")
pprint.pprint(feasibility_complexity)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' A specific evaluation request with clear hardware parameters and an analysis requirement focused on deterministic performance. ': {'count': 1,
                                                                                                                                              'indices': [264]},
         ' A specific evaluation request with clear hardware parameters and an analysis requirement. ': {'count': 1,
                                                                                                         'indices': [214]},
         ' A specific hardware evaluation request involving specialized storage/memory technology. ': {'count': 1,
                                                                                                       'indices': [327]},
         ' A specific hardware evaluation request. ': {'count': 12,
                                                       'indices': [222,
                               

In [82]:
feasibility_complexity = get_intent_justification_details(df, "Intent Feasibility Check", "Complexity (1-5)")
pprint.pprint(feasibility_complexity)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' A specific evaluation request with clear hardware parameters and an analysis requirement focused on deterministic performance. ': {'count': 1,
                                                                                                                                              'indices': [264]},
         ' A specific evaluation request with clear hardware parameters and an analysis requirement. ': {'count': 1,
                                                                                                         'indices': [214]},
         ' A specific hardware evaluation request involving specialized storage/memory technology. ': {'count': 1,
                                                                                                       'indices': [327]},
         ' A specific hardware evaluation request. ': {'count': 12,
                                                       'indices': [222,
                               

### **Standards Alignment / Correctness (1-5)**

In [83]:
feasibility_standards_alignment = get_intent_justification_details(df, "Intent Feasibility Check", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(feasibility_standards_alignment)

{'0–1': {},
 '1–2': {},
 '2–3': {" This contains a standards error. 3GPP's NEA1 algorithm is 128-bit, not 256-bit. EAP-TLS is a valid authentication method. High-density support is a valid 5G concept (mMTC). ": {'count': 1,
                                                                                                                                                                                            'indices': [3023]}},
 '3–4': {' "High-high-throughput" is redundant and awkward. "Deployment complexity" is a subject of the analysis, not a characteristic of the architecture itself in this context. ': {'count': 1,
                                                                                                                                                                                       'indices': [10]},
         ' "High-high-throughput" is redundant. "Hybrid_4G_5G" is not a standard term; the correct term would be 5G Non-Standalone (NSA) or E-UTRA-NR Dual Connectivity (EN

### **Readability (1-5)**

In [84]:
feasibility_readability = get_intent_justification_details(df, "Intent Feasibility Check", 'Readability (1-5)')
pprint.pprint(feasibility_readability)

{'0–1': {},
 '1–2': {},
 '2–3': {' The repetitive use of "high-priority" and the redundant "high-high-throughput" makes the description cluttered and hard to read. ': {'count': 1,
                                                                                                                                                'indices': [75]}},
 '3–4': {' The readability is slightly impaired by the repetitive "ultra-ultra-reliable low-ultra-low latency" and the somewhat clunky phrasing at the end. ': {'count': 1,
                                                                                                                                                                'indices': [2294]},
         ' The redundant phrasing and non-standard terminology ("Hybrid_4G_5G") reduce readability and clarity. ': {'count': 1,
                                                                                                                    'indices': [35]},
         ' The redundant phrasing reduces r

### **Relevance (1-5)**

In [85]:
feasibility_relevance = get_intent_justification_details(df, "Intent Feasibility Check", 'Relevance (1-5)')
pprint.pprint(feasibility_relevance)

{'0–1': {}, '1–2': {}, '2–3': {}, '3–4': {}}


### **Coherence (1-5)**

In [86]:
feasibility_coherence = get_intent_justification_details(df, "Intent Feasibility Check", 'Coherence (1-5)')
pprint.pprint(feasibility_coherence)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' The structure is logical, but the hardware combination is so imbalanced it borders on incoherent. ': {'count': 1,
                                                                                                                 'indices': [3160]}}}


### **Consistency (1-5)**

In [87]:
feasibility_consistency = get_intent_justification_details(df, "Intent Feasibility Check", 'Consistency (1-5)')
pprint.pprint(feasibility_consistency)

{'0–1': {},
 '1–2': {},
 '2–3': {' A 64.03ms latency is inconsistent with the descriptor "ultra-low latency." This is a significant contradiction. ': {'count': 1,
                                                                                                                               'indices': [3780]},
         ' The use of "256_NEA1" is inconsistent with the 3GPP standard for the NEA1 algorithm, creating a technical contradiction. ': {'count': 1,
                                                                                                                                        'indices': [3023]},
         ' There is a major performance inconsistency. Any workload needing HBM2 would be severely bottlenecked by SATA SSDs. ': {'count': 1,
                                                                                                                                  'indices': [3160]}},
 '3–4': {' For a "mission-critical" system, pairing high-speed DDR5 memory with slower SATA S

### **Completeness (1-5)**"""

In [88]:
feasibility_completeness = get_intent_justification_details(df, "Intent Feasibility Check", 'Completeness (1-5)')
pprint.pprint(feasibility_completeness)

{'0–1': {},
 '1–2': {},
 '2–3': {' The intent is critically incomplete as it does not specify which application or network function the hardware is intended for. ': {'count': 13,
                                                                                                                                              'indices': [3630,
                                                                                                                                                          3641,
                                                                                                                                                          3643,
                                                                                                                                                          3653,
                                                                                                                                                          3681,
                     

### **Overall Score (1-5)**"""

In [89]:
feasibility_overall_score = get_intent_justification_details(df, "Intent Feasibility Check", 'Overall Score')
pprint.pprint(feasibility_overall_score)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' A clear feasibility check that is weakened by a questionable and imbalanced hardware configuration and a lack of context.': {'count': 1,
                                                                                                                                        'indices': [3092]},
         ' A moderately well-formed intent that is weakened by a lack of workload context and an inconsistent hardware specification for the stated criticality.': {'count': 1,
                                                                                                                                                                    'indices': [4511]},
         ' A moderately well-formed intent that is weakened by a lack of workload context and an inconsistent hardware specification.': {'count': 1,
                                                                                                                                         'indices': [44

## Intent Report Request Analysis

### **Complexity Measure (1-5)**
"""

In [39]:
report_request_complexity = get_intent_justification_details(df, "Intent Report Request", "Complexity (1-5)")
pprint.pprint(report_request_complexity)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' A standard report request for a high-resource NF instance. ': {'count': 1,
                                                                          'indices': [4754]},
         ' A standard report request for a relatively low-resource NF instance. ': {'count': 1,
                                                                                    'indices': [4749]},
         ' A standard report request for a very high-resource NF instance. ': {'count': 1,
                                                                               'indices': [4736]},
         ' A standard report request with a high-importance qualifier for a high-resource NF instance. ': {'count': 1,
                                                                                                           'indices': [4787]},
         ' A standard report request with a priority level for a high-resource NF instance. ': {'count': 1,
                                        

### **Standards Alignment / Correctness (1-5)**"""

In [40]:
report_request_standards_alignment = get_intent_justification_details(df, "Intent Report Request", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(report_request_standards_alignment)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' The application of ML models like Isolation Forest for anomaly detection is consistent with NWDAF principles (TS 23.288), but the model and parameters are not 3GPP standards. ': {'count': 2,
                                                                                                                                                                                              'indices': [2237,
                                                                                                                                                                                                          2351]},
         ' The application of ML models like LSTM Autoencoders for anomaly detection aligns with NWDAF concepts (TS 23.288). However, the specific model and parameters (sampling, retention) are implementation details, not 3GPP standards. ': {'count': 1,
                                                                                          

### **Readability (1-5)**"""

In [41]:
report_request_readability = get_intent_justification_details(df, "Intent Report Request", 'Readability (1-5)')
pprint.pprint(report_request_readability)

{'0–1': {}, '1–2': {}, '2–3': {}, '3–4': {}}


### **Relevance (1-5)**"""

In [42]:
report_request_relevance = get_intent_justification_details(df, "Intent Report Request", 'Relevance (1-5)')
pprint.pprint(report_request_relevance)

{'0–1': {}, '1–2': {}, '2–3': {}, '3–4': {}}


### **Coherence (1-5)**"""

In [43]:
report_request_coherence = get_intent_justification_details(df, "Intent Report Request", 'Coherence (1-5)')
pprint.pprint(report_request_coherence)

{'0–1': {}, '1–2': {}, '2–3': {}, '3–4': {}}


### **Consistency (1-5)**"""

In [44]:
report_request_consistency = get_intent_justification_details(df, "Intent Report Request", 'Consistency (1-5)')
pprint.pprint(report_request_consistency)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' The extreme and imbalanced resource values (12 cores, 508GB RAM) are inconsistent with typical server configurations. ': {'count': 1,
                                                                                                                                     'indices': [107]},
         ' The extreme and imbalanced resource values (128 cores, 14GB RAM) are inconsistent with typical server configurations for a network function like AMF. ': {'count': 1,
                                                                                                                                                                     'indices': [47]},
         ' The resource allocation of 111 cores to only 29GB of memory is a highly unusual and imbalanced ratio, suggesting a potential error or a very niche workload. ': {'count': 1,
                                                                                                                           

### **Completeness (1-5)**"""

In [45]:
report_request_completeness = get_intent_justification_details(df, "Intent Report Request", 'Completeness (1-5)')
pprint.pprint(report_request_completeness)

{'0–1': {},
 '1–2': {},
 '2–3': {' The intent is critically incomplete. It specifies the "how" (ML model, data parameters) but not the "what" (the subject of the analysis, e.g., AMF performance, RAN congestion). ': {'count': 1,
                                                                                                                                                                                                'indices': [940]}},
 '3–4': {" The intent is critically incomplete as it doesn't specify which data or metrics are to be analyzed. ": {'count': 4,
                                                                                                                   'indices': [73,
                                                                                                                               76,
                                                                                                                               87,
                                        

### **Overall Score (1-5)**"""

In [46]:
report_request_overall_score = get_intent_justification_details(df, "Intent Report Request", 'Overall Score')
pprint.pprint(report_request_overall_score)

{'0–1': {}, '1–2': {}, '2–3': {}, '3–4': {}}


## Modification Intent Analysis

### **Complexity Measure (1-5)**
"""

In [47]:
modification_complexity = get_intent_justification_details(df, "Modification Intent", "Complexity (1-5)")
pprint.pprint(modification_complexity)

{'0–1': {},
 '1–2': {},
 '2–3': {' The intent specifies core/memory numbers but then adds a vague, unrelated action ("with transport network optimization") without any details. ': {'count': 2,
                                                                                                                                                             'indices': [2070,
                                                                                                                                                                         2139]}},
 '3–4': {' A moderately complex intent involving resource allocation changes tied to a broad optimization goal. ': {'count': 1,
                                                                                                                    'indices': [4784]},
         ' A moderately complex intent involving significant resource allocation changes tied to a broad optimization goal. ': {'count': 1,
                                                  

### **Standards Alignment / Correctness (1-5)**"""

In [48]:
modification_standards_alignment = get_intent_justification_details(df, "Modification Intent", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(modification_standards_alignment)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' Mixes generic infrastructure terms (cores, memory) with a high-level network goal. "Core network" is not specific. ': {'count': 3,
                                                                                                                                  'indices': [4853,
                                                                                                                                              4928,
                                                                                                                                              4945]},
         ' Similar to previous examples, this intent ambiguously links core compute resource modification with a vague "transport network optimization" goal. The connection is not explicit. ': {'count': 2,
                                                                                                                                                                          

### **Readability (1-5)**"""

In [49]:
modification_readability = get_intent_justification_details(df, "Modification Intent", 'Readability (1-5)')
pprint.pprint(modification_readability)

{'0–1': {},
 '1–2': {},
 '2–3': {" The phrasing is disjointed. It's unclear if the resource modification is FOR the transport optimization or a separate action. ": {'count': 2,
                                                                                                                                             'indices': [92,
                                                                                                                                                         126]},
         ' The readability is poor due to awkwardly formatted model names ("lstm_with high-priority_attention_mechanism") and redundant priority flags. ': {'count': 2,
                                                                                                                                                            'indices': [2241,
                                                                                                                                                                    

### **Relevance (1-5)**"""

In [50]:
modification_relevance = get_intent_justification_details(df, "Modification Intent", 'Relevance (1-5)')
pprint.pprint(modification_relevance)

{'0–1': {},
 '1–2': {' The description is about monitoring and analysis, not modification. There is a clear mismatch with the intent type. ': {'count': 1,
                                                                                                                                   'indices': [28]}},
 '2–3': {},
 '3–4': {}}


### **Coherence (1-5)**"""

In [51]:
modification_coherence = get_intent_justification_details(df, "Modification Intent", 'Coherence (1-5)')
pprint.pprint(modification_coherence)

{'0–1': {},
 '1–2': {},
 '2–3': {' The coherence is low. A direct causal link between adjusting compute on an unspecified NF and "transport network optimization" is not clearly established. ': {'count': 3,
                                                                                                                                                                          'indices': [3448,
                                                                                                                                                                                      3495,
                                                                                                                                                                                      3517]},
         ' The coherence is low. Modifying compute resources in the core network is not a direct way to optimize the transport network; they are separate domains. ': {'count': 4,
                                                 

### **Consistency (1-5)**"""

In [52]:
modification_consistency = get_intent_justification_details(df, "Modification Intent", 'Consistency (1-5)')
pprint.pprint(modification_consistency)

{'0–1': {},
 '1–2': {' The intent is highly inconsistent. 5QI 85 is designed for low-bandwidth, high-reliability SCADA-type traffic. A bitrate of 4078 Mbps is completely inappropriate and contradicts the purpose of this standardized 5QI. ': {'count': 1,
                                                                                                                                                                                                                                      'indices': [3244]},
         ' The requested maximum bitrate of over 6 Gbps is completely inconsistent with the nature of a conversational voice service (which is typically <100 kbps). This makes the intent nonsensical. ': {'count': 1,
                                                                                                                                                                                                            'indices': [2511]},
         ' The specified bitrate of over 9Gbps for a voi

### **Completeness (1-5)**"""

In [53]:
modification_completeness = get_intent_justification_details(df, "Modification Intent", 'Completeness (1-5)')
pprint.pprint(modification_completeness)

{'0–1': {},
 '1–2': {},
 '2–3': {' The intent is critically incomplete. It fails to specify which Network Function(s) in the core network should be modified. ': {'count': 5,
                                                                                                                                          'indices': [921,
                                                                                                                                                      951,
                                                                                                                                                      981,
                                                                                                                                                      989,
                                                                                                                                                      996]},
         " The intent is highly incomplete. It do

### **Overall Score (1-5)**"""

In [54]:
modification_overall_score = get_intent_justification_details(df, "Modification Intent", 'Overall Score')
pprint.pprint(modification_overall_score)

{'0–1': {},
 '1–2': {},
 '2–3': {' A poorly constructed intent that is vague, incoherent, and incomplete. The link between the two clauses is not explained.': {'count': 2,
                                                                                                                                        'indices': [2230,
                                                                                                                                                    2267]}},
 '3–4': {' A clear resource modification intent that is significantly weakened by the inclusion of a vague and poorly connected secondary goal.': {'count': 6,
                                                                                                                                                   'indices': [3226,
                                                                                                                                                               3241,
                            

## Regular Notification Request Analysis
### **Complexity Measure (1-5)**
"""

In [55]:
regular_notification_complexity = get_intent_justification_details(df, "Regular Notification Request", "Complexity (1-5)")
pprint.pprint(regular_notification_complexity)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' A specific but standard monitoring and alerting rule, making it moderately complex. ': {'count': 4,
                                                                                                   'indices': [1423,
                                                                                                               1444,
                                                                                                               1499,
                                                                                                               1525]},
         ' A specific but standard monitoring and alerting rule, with a very frequent (1 hour) key rotation context. ': {'count': 1,
                                                                                                                         'indices': [1547]},
         ' Defines a notification based on two performance thresholds (throughput, latency) and specifies a f

### **Standards Alignment / Correctness (1-5)**"""

In [56]:
regular_notification_standards_alignment = get_intent_justification_details(df, "Regular Notification Request", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(regular_notification_standards_alignment)

{'0–1': {},
 '1–2': {},
 '2–3': {' 5G_AKA is a correct authentication method. However, `256_NEA1` is non-standard terminology, as 3GPP NAS encryption algorithms (NEA) are 128-bit. ': {'count': 2,
                                                                                                                                                                'indices': [807,
                                                                                                                                                                            852]},
         ' 5G_AKA is a correct authentication method. However, `256_NEA2` is non-standard terminology, as 3GPP NAS encryption algorithms (NEA) are 128-bit. ': {'count': 1,
                                                                                                                                                                'indices': [930]},
         ' Certificate-based auth is valid. However, 128_NEA1 does not exist; NEA1 is NULL encryp

### **Readability (1-5)**"""

In [57]:
regular_notification_readability = get_intent_justification_details(df, "Regular Notification Request", 'Readability (1-5)')
pprint.pprint(regular_notification_readability)

{'0–1': {},
 '1–2': {},
 '2–3': {' The extremely repetitive and redundant phrasing makes the description difficult to read and unprofessional. ': {'count': 1,
                                                                                                                           'indices': [141]},
         ' The intent is readable, but the glaring technical inaccuracy about NEA1 makes it confusing and incorrect. ': {'count': 6,
                                                                                                                         'indices': [83,
                                                                                                                                     120,
                                                                                                                                     121,
                                                                                                                                     138,
               

### **Relevance (1-5)**"""

In [58]:
regular_notification_relevance = get_intent_justification_details(df, "Regular Notification Request", 'Relevance (1-5)')
pprint.pprint(regular_notification_relevance)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {" This is a request for alerting, but it's framed as a configuration/deployment of a monitoring system. The intent type and description are not perfectly aligned. ": {'count': 2,
                                                                                                                                                                                'indices': [854,
                                                                                                                                                                                            877]},
         ' This is a request for alerts, but it\'s based on a vague "insights" condition, making it hard to implement as a standard notification request. ': {'count': 1,
                                                                                                                                                              'indices': [945]},
         ' This is more of a `Performance

### **Coherence (1-5)**"""

In [59]:
regular_notification_coherence = get_intent_justification_details(df, "Regular Notification Request", 'Coherence (1-5)')
pprint.pprint(regular_notification_coherence)

{'0–1': {},
 '1–2': {},
 '2–3': {' The coherence is low because Simulated Annealing is an optimization method, not a data analysis or anomaly detection method for generating alerts. ': {'count': 1,
                                                                                                                                                                  'indices': [1031]},
         ' The coherence is low because the chosen algorithm (Genetic Algorithm) is not appropriate for the stated task (trend analysis for notifications). ': {'count': 2,
                                                                                                                                                                'indices': [1123,
                                                                                                                                                                            1165]},
         ' The coherence is low because the chosen algorithm (Particle Swarm) is not appro

### **Consistency (1-5)**"""

In [60]:
regular_notification_consistency = get_intent_justification_details(df, "Regular Notification Request", 'Consistency (1-5)')
pprint.pprint(regular_notification_consistency)

{'0–1': {},
 '1–2': {' The intent is fundamentally inconsistent as it refers to a non-existent security algorithm (128-bit NEA1). ': {'count': 5,
                                                                                                                          'indices': [83,
                                                                                                                                      120,
                                                                                                                                      138,
                                                                                                                                      149,
                                                                                                                                      184]},
         ' The intent is fundamentally inconsistent as it refers to a non-existent security algorithm (256-bit NEA1). ': {'count': 1,
                        

### **Completeness (1-5)**"""

In [61]:
regular_notification_completeness = get_intent_justification_details(df, "Regular Notification Request", 'Completeness (1-5)')
pprint.pprint(regular_notification_completeness)

{'0–1': {},
 '1–2': {' This intent is critically incomplete. It fails to specify what trend or condition should trigger the alert. "Intelligent insights" is not a measurable trigger. ': {'count': 1,
                                                                                                                                                                               'indices': [945]}},
 '2–3': {' Like record #1, this intent is critically incomplete as it fails to specify the trigger for the notification. ': {'count': 1,
                                                                                                                             'indices': [5]},
         ' The intent is missing the crucial trigger condition for the notification, making it incomplete. ': {'count': 2,
                                                                                                               'indices': [0,
                                                                              

### **Overall Score (1-5)**"""

In [62]:
regular_notification_overall_score = get_intent_justification_details(df, "Regular Notification Request", 'Overall Score')
pprint.pprint(regular_notification_overall_score)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' A conceptually flawed and abstract intent. The choice of algorithm is questionable for the stated purpose, and it lacks actionable detail.': {'count': 2,
                                                                                                                                                         'indices': [2394,
                                                                                                                                                                     2404]},
         ' A conceptually interesting but abstract intent that lacks the specific metrics needed to be actionable.': {'count': 3,
                                                                                                                      'indices': [2221,
                                                                                                                                  2307,
                                                     

## Performance Assurance Intent Analysis

### **Complexity Measure (1-5)**
"""

In [63]:
performance_assurance_complexity = get_intent_justification_details(df, "Performance Assurance Intent", "Complexity (1-5)")
pprint.pprint(performance_assurance_complexity)

{'0–1': {},
 '1–2': {},
 '2–3': {' The intent is extremely high-level and strategic, lacking any specific technical details or measurable outcomes. ': {'count': 1,
                                                                                                                                'indices': [477]},
         " The intent is very high-level and abstract, lacking any specific technical details or metrics. It's a strategic goal, not a detailed technical intent. ": {'count': 1,
                                                                                                                                                                      'indices': [4293]},
         ' The intent is very simple, stating a high-level, aspirational goal without any concrete parameters. ': {'count': 3,
                                                                                                                   'indices': [2242,
                                                                 

### **Standards Alignment / Correctness (1-5)**"""

In [64]:
performance_assurance_standards_alignment = get_intent_justification_details(df, "Performance Assurance Intent", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(performance_assurance_standards_alignment)

{'0–1': {},
 '1–2': {},
 '2–3': {' 5G_AKA and automated key management are correct. However, `256_NEA2` is non-standard terminology, as 3GPP NEA algorithms are 128-bit. "Performance assurance" is also misused; this is a security intent. ': {'count': 1,
                                                                                                                                                                                                                         'indices': [841]},
         ' Certificate-based authentication is valid. However, `256_NEA1` is non-standard terminology; 3GPP NEA algorithms are 128-bit. The typo "high-priorityout" also reduces quality. ': {'count': 1,
                                                                                                                                                                                              'indices': [827]},
         ' EAP-AKA\' is a standard authentication method. However, NEA1 is NULL encryption. Using it

### **Readability (1-5)**"""

In [65]:
performance_assurance_readability = get_intent_justification_details(df, "Performance Assurance Intent", 'Readability (1-5)')
pprint.pprint(performance_assurance_readability)

{'0–1': {},
 '1–2': {' The description is riddled with technical inaccuracies (NEA1 definition) and a typo ("priorityout"), making it very difficult to read correctly. ': {'count': 1,
                                                                                                                                                                'indices': [39]}},
 '2–3': {' The description is a dense string of technical jargon ("cognitive," "fault-tolerant," "deep learning") that harms clarity. ': {'count': 2,
                                                                                                                                          'indices': [11,
                                                                                                                                                      191]},
         ' The description is highly redundant and awkwardly phrased ("multi_objective_genetic_algorithm multi-objective optimization"), which significantly harms readability. 

### **Relevance (1-5)**"""

In [66]:
performance_assurance_relevance = get_intent_justification_details(df, "Performance Assurance Intent", 'Relevance (1-5)')
pprint.pprint(performance_assurance_relevance)

{'0–1': {},
 '1–2': {' The intent describes security implementation (authentication, encryption, key management), not performance assurance (e.g., latency, throughput). This is a severe mismatch. ': {'count': 1,
                                                                                                                                                                                            'indices': [841]},
         ' The intent is about implementing security policies (authentication, encryption), not assuring performance metrics (latency, throughput). This is a severe mismatch. ': {'count': 1,
                                                                                                                                                                                   'indices': [922]}},
 '2–3': {" This intent is about implementing security policies, not assuring performance metrics. There is a significant mismatch between the intent type and the description's focus. ": {'cou

### **Coherence (1-5)**"""

In [67]:
performance_assurance_coherence = get_intent_justification_details(df, "Performance Assurance Intent", 'Coherence (1-5)')
pprint.pprint(performance_assurance_coherence)

{'0–1': {},
 '1–2': {},
 '2–3': {' The components are related to security, but the incorrect use of terms makes the intent incoherent. ': {'count': 1,
                                                                                                                   'indices': [39]},
         ' The intent is incoherent because it proposes to achieve security by using a non-existent encryption algorithm based on NULL encryption. ': {'count': 1,
                                                                                                                                                       'indices': [170]},
         ' The intent is incoherent because it proposes to achieve security by using a null encryption algorithm. ': {'count': 1,
                                                                                                                      'indices': [44]}},
 '3–4': {' The clumsy formatting of the model name and redundant keywords hurt the coherence of the sentence. ': {'c

### **Consistency (1-5)**"""

In [68]:
performance_assurance_consistency = get_intent_justification_details(df, "Performance Assurance Intent", 'Consistency (1-5)')
pprint.pprint(performance_assurance_consistency)

{'0–1': {},
 '1–2': {' The intent is fundamentally inconsistent because 256-NEA2 is not a valid 3GPP algorithm. ': {'count': 1,
                                                                                                        'indices': [140]},
         ' The intent is fundamentally inconsistent, as it aims to "secure" infrastructure using NULL (no) encryption. ': {'count': 1,
                                                                                                                           'indices': [39]},
         ' The intent is fundamentally inconsistent, as the action (use 256_NEA1) is impossible and contradicts the goal (secure... infrastructure). ': {'count': 1,
                                                                                                                                                         'indices': [170]},
         ' The intent is fundamentally inconsistent, as the action (use NEA1) directly contradicts the goal (secure... infrastructure). 

### **Completeness (1-5)**"""

In [69]:
performance_assurance_completeness = get_intent_justification_details(df, "Performance Assurance Intent", 'Completeness (1-5)')
pprint.pprint(performance_assurance_completeness)

{'0–1': {},
 '1–2': {" The intent is completely lacking in specific, actionable details. It's a goal, not a command. ": {'count': 1,
                                                                                                             'indices': [1363]},
         ' The intent is critically incomplete. It describes a method but fails to specify which performance objectives (e.g., latency vs. throughput vs. cost) are being traded off. ': {'count': 1,
                                                                                                                                                                                          'indices': [995]},
         ' The intent is entirely incomplete. It states a laudable goal but provides no metrics, targets, services, or methods. It is not actionable. ': {'count': 1,
                                                                                                                                                          'indices': [370

### **Overall Score (1-5)**"""

In [70]:
performance_assurance_overall_score = get_intent_justification_details(df, "Performance Assurance Intent", 'Overall Score')
pprint.pprint(performance_assurance_overall_score)

{'0–1': {},
 '1–2': {},
 '2–3': {' A critically flawed intent due to a fundamental misunderstanding of 3GPP security algorithms, specifically NEA1.': {'count': 1,
                                                                                                                               'indices': [44]},
         ' A critically flawed intent due to a fundamental misunderstanding of 3GPP security terminology (NEA1).': {'count': 1,
                                                                                                                    'indices': [170]},
         ' A critically flawed intent due to fundamental misunderstandings of 3GPP security terminology (NEA1).': {'count': 1,
                                                                                                                   'indices': [39]}},
 '3–4': {' A coherent intent that is weakened by a non-standard algorithm and is fundamentally incomplete as a performance intent due to the absence of metrics.': {'cou

##Deployment Intent Analysis

### **Complexity Measure (1-5)**
"""

In [71]:
deployment_complexity = get_intent_justification_details(df, "Deployment Intent", "Complexity (1-5)")
pprint.pprint(deployment_complexity)

{'0–1': {},
 '1–2': {},
 '2–3': {" It's a simple resource allocation request, specifying only CPU cores and memory. ": {'count': 5,
                                                                                                'indices': [4813,
                                                                                                            4897,
                                                                                                            4927,
                                                                                                            4939,
                                                                                                            4978]},
         " It's a simple resource request, specifying only core count, architecture, and memory. ": {'count': 5,
                                                                                                     'indices': [4832,
                                                                

### **Standards Alignment / Correctness (1-5)**"""

In [72]:
deployment_standered_allignment= get_intent_justification_details(df, "Deployment Intent", 'Standards Alignment / Correctness (1-5)')
pprint.pprint(deployment_standered_allignment)

{'0–1': {},
 '1–2': {},
 '2–3': {' Hardware-based attestation is a valid concept. However, 256_NEA2 does not exist; NEA2 (AES) is a 128-bit algorithm in 3GPP. "hardware_based_attestation hardware attestation" is redundant. ': {'count': 1,
                                                                                                                                                                                                           'indices': [181]},
         ' NSA and 5G_AKA are standard. However, `256_NEA1` is non-standard terminology. Also, the primary authentication for NSA typically relies on the 4G EPC (EPS-AKA). 5G_AKA is for 5G core (SA). ': {'count': 1,
                                                                                                                                                                                                            'indices': [964]},
         ' There is a major contradiction. A service cannot be both "low priority" and support "miss

### **Readability (1-5)**"""

In [73]:
deployment_readability = get_intent_justification_details(df, "Deployment Intent", "Readability (1-5)")
pprint.pprint(deployment_readability)

{'0–1': {},
 '1–2': {' The excessive and repetitive use of adjectives makes the sentence almost unreadable and highly unprofessional. It\'s a classic example of "buzzword stuffing." ': {'count': 1,
                                                                                                                                                                              'indices': [31]},
         ' The intent is nearly unreadable due to the excessive and awkwardly placed repetition of adjectives like "research-grade sophisticated" and "mission-critical". ': {'count': 2,
                                                                                                                                                                              'indices': [4418,
                                                                                                                                                                                          4436]},
         ' The intent is severely bloa

### **Relevance (1-5)**"""

In [74]:
deployment_relevance = get_intent_justification_details(df, "Deployment Intent", "Relevance (1-5)")
pprint.pprint(deployment_relevance)

{'0–1': {},
 '1–2': {' The description is about monitoring and analysis, not deployment. There is a clear mismatch with the intent type. ': {'count': 2,
                                                                                                                                 'indices': [82,
                                                                                                                                             113]},
         ' The description is about monitoring and responding to events, not deploying a new system. There is a clear mismatch. ': {'count': 1,
                                                                                                                                    'indices': [97]}},
 '2–3': {' The description is about optimizing an existing service, which is not a deployment task. ': {'count': 1,
                                                                                                        'indices': [109]},
         ' This desc

### **Coherence (1-5)**"""

In [76]:
deployment_coherence = get_intent_justification_details(df, "Deployment Intent", "Coherence (1-5)")
pprint.pprint(deployment_coherence)

{'0–1': {},
 '1–2': {' The sentence structure is broken and convoluted by the redundant adjectives, making it incoherent and difficult to follow. ': {'count': 1,
                                                                                                                                          'indices': [2211]},
         ' The sentence structure is broken and incoherent due to the repetitive, meaningless adjectives. ': {'count': 1,
                                                                                                              'indices': [4436]},
         ' The sentence structure is broken and incoherent due to the repetitive, non-technical adjectives. ': {'count': 1,
                                                                                                                'indices': [4418]}},
 '2–3': {" The core technical request is coherent, but it's obscured by layers of repetitive, unstructured adjectives that disrupt the logical flow. ": {'count': 1,
      

### **Consistency (1-5)**"""

In [77]:
deployment_consistency = get_intent_justification_details(df, "Deployment Intent", "Consistency (1-5)")
pprint.pprint(deployment_consistency)

{'0–1': {},
 '1–2': {' Requesting "0 processing cores" is illogical and inconsistent. A computing resource cannot function without at least one core. This is a significant correctness flaw. ': {'count': 1,
                                                                                                                                                                                      'indices': [2739]},
         ' The intent is fundamentally inconsistent because 256-NEA2 is not a valid 3GPP algorithm. ': {'count': 1,
                                                                                                        'indices': [181]},
         ' The intent is fundamentally inconsistent by mixing "low priority" with "mission-critical," "strict" delay, and "deterministic" performance. ': {'count': 1,
                                                                                                                                                           'indices': [8]},
         ' The

### **Completeness (1-5)**"""

In [78]:
deployment_completeness = get_intent_justification_details(df, "Deployment Intent", "Completeness (1-5)")
pprint.pprint(deployment_completeness)

{'0–1': {},
 '1–2': {},
 '2–3': {' The intent is critically incomplete. It provides a detailed description of the deployment process but fails to mention what is being deployed. ': {'count': 1,
                                                                                                                                                              'indices': [971]},
         ' The intent is highly incomplete. It does not specify which network function(s) these resources are for, which is essential information. ': {'count': 4,
                                                                                                                                                       'indices': [3535,
                                                                                                                                                                   3577,
                                                                                                                                

### **Overall Score (1-5)**"""

In [79]:
deployment_overall_score = get_intent_justification_details(df, "Deployment Intent", "Overall Score")
pprint.pprint(deployment_overall_score)

{'0–1': {},
 '1–2': {},
 '2–3': {},
 '3–4': {' A conceptually valid but poorly written intent that uses excessive jargon and non-standard terms, and is too high-level to be actionable.': {'count': 1,
                                                                                                                                                        'indices': [2392]},
         ' A relevant but poorly phrased intent with potentially conflicting performance requirements that would need clarification.': {'count': 1,
                                                                                                                                        'indices': [4407]},
         ' A syntactically correct intent that is rendered invalid and inconsistent by the illogical request for zero processing cores.': {'count': 1,
                                                                                                                                           'indices': [2739]},
         ' A t